In [2]:
# 必要ライブラリ
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean, median,variance,stdev
import datetime as dt
import burst_detection as bd
import numpy as np

In [3]:
#テキスト日付をdate日付にする
def text_date_to_date(l):
    if l[0] <= 9:
        m = '0' + str( l[0] )
    else:
        m = str( l[0] )
    if l[1] <= 9:
        d = '0' + str( l[1] )
    else:
        d = str( l[1] )
    
    d = m + '/' + d + '/2015'
    date = dt.datetime.strptime(d, '%m/%d/%Y').date()
    return(date)

In [4]:
def convert_list(lines):
    list_d = []
    for line in lines:
        l_list = line.split('\t')
        list_d.append( [ int(l_list[0]), int(l_list[1]), int(l_list[2]) ] )
        
    return(list_d)

In [5]:
def date_hit_list(list_d):
    list_date_hit = []
    dates = []
    hits = []
    for i, l in enumerate(list_d):
        date = text_date_to_date(l) 
        dates.append( date )
        hits.append( l[2] )
    list_date_hit.append(dates)
    list_date_hit.append(hits)
    return(list_date_hit)

In [6]:
def daterange(start, end):
  for n in range((end - start).days):
    yield start + timedelta(n)

## kleinbergのバースト

In [11]:
# 検出したバーストをプロットする
def plot_burst(q, dates):
    correct = []
    for date in dates:
        if date > CORRECT_START and date < CORRECT_END:
            correct.append(1)
        else:
            correct.append(0)
    y_correct = np.array(correct[20:90])

    x_date = np.array(dates[20:90])
    y_burst = q.T[0][20:90]

    fig = plt.figure(figsize=(10,4),dpi=150)
    ax = fig.add_subplot(1, 1, 1)
    plt.tight_layout()
    plt.yticks( [0, 1] )
    ax.xaxis.set_major_locator(mdates.DayLocator(bymonthday=None, interval=7, tz=None))
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%m/%d"))

    plt.plot(x_date, y_burst, linewidth=3.0)
    plt.plot(x_date, y_correct, linewidth=3.0)

    #plt.xlabel('date')
    plt.ylabel('burst')

    plt.tight_layout()

    ax.tick_params(axis='both',labelsize=18)
    fig.autofmt_xdate()
    fig.text(0.12, 0.4, "rate = " + str(rate), fontsize=30)

    # 保存
    pngname = 'hk_burst.png'
    plt.savefig(pngname)

    plt.show()

In [29]:
千# バースト検出
def burst_detection(list_d, list_d_all, dates):
    #x軸を作成（日付），y軸を作成（件数），
    d = date_hit_list(list_d)
    dates = d[0]
    hits = d[1]

    d_all = date_hit_list(list_d_all)
    # dates_all = date_hit_list_all[0]
    hits_all = d_all[1]
    
    #number of target events at each time point
    r = np.array(hits, dtype=float)
    #total number of events at each time point
    d = np.array(hits_all, dtype=float)
    #number of time points
    n = len(r)

    q, d, r, p = bd.burst_detection(r,d,n,s=2,gamma=1,smooth_win=1)
    bursts = bd.enumerate_bursts(q, 'burstLabel')

    #find weight of bursts
    weighted_bursts = bd.burst_weights(bursts,r,d,p)

    print('observed probabilities: ')
    # print(str(r/d) )

    print( 'optimal state sequence: ')
    print(str(q.T) )

    print( 'baseline probability: ' + str(p[0]) )

    print( 'bursty probability: ' + str(p[1]) )

    print( 'weighted bursts:' )
    print( weighted_bursts)
    print(weighted_bursts.iloc[2, 1])
    
    plot_burst(q, dates)
    
    return(weighted_bursts)

In [10]:
# バーストと正解期間から求めたF値をプロット
def eva_plot(fscore):
    fig = plt.figure(figsize=(10,10),dpi=150)
    ax = fig.add_subplot(1, 1, 1)

    x = [], y = []
    
    y.append(fscore)
    x.append(rate)

    ax_x = np.array(x)
    ax_y = np.array(y)

    label = 'Relation'
    plt.plot(ax_x, ax_y, label = label,  linewidth = 3.0)  
    plt.tight_layout()
    plt.rcParams["font.size"] = 18

    plt.xlabel('percent')
    plt.ylabel('F-score')

    ax.tick_params(axis='both',labelsize=18)
    fig.autofmt_xdate()
    
    # 保存
    plt.savefig(pid + "_relation_fscore.pdf")

    plt.show()

In [30]:
#　検出したバースト期間の評価
def eva(dates, correct, weighted_bursts):
    corrects = dates[CORRECT_START:CORRECT_END] 
    bursts = []
    for i in range(0, weighted_bursts):
        bursts.extend(dates[[weighted_bursts.iloc[i, 1]] : [weighted_bursts.iloc[i, 2]]])
                            
    period = corrects + bursts
    START_DATE = min(period)
    END_DATE = max(period)

    tp, fn, fp = 0, 0, 0
    for date in daterange(START_DATE, END_DATE):
        if date in corrects :
            if date in bursts:
                tp = tp + 1
            else :
                fn = fn + 1
        elif date in bursts:
            fp = fp + 1
            
    rec = tp / (tp + fn)
    pre = tp / (tp + fp)
    fscore = (2 * rec * pre) /(rec + pre)
    
    # 評価値を出す
    print('rec', rec)
    print('pre', pre)
    print('F-score', fscore)
    
    plot_eva(fscore)
    
    return

In [9]:
# データ準備

sakura_tweets = "hk_sakura_cleaning.txt"
all_tweets = "hk_all_cleaning.txt"

#exts = 'soa'
#rates = range(10, 101, 10)

# correct: 桜の見頃正解期間（開花日〜満開日）
CORRECT_START = text_date_to_date([4,30])
CORRECT_END =  text_date_to_date([5,5])

fp_h = open(sakura_tweets, "r")
fp_h_all = open(all_tweets, "r")
line = fp_h.readlines()
line_all = fp_h_all.readlines()

list_d = convert_list(line)
list_d_all = convert_list(line_all)

#x軸を作成（日付），y軸を作成（件数），
d = date_hit_list(list_d)
dates = d[0]
hits = d[1]

d_all = date_hit_list(list_d_all)
# dates_all = date_hit_list_all[0]
hits_all = d_all[1]

fp_h.close()
fp_h_all.close()


FileNotFoundError: [Errno 2] No such file or directory: 'hk_sakura_cleaning.txt'

In [1]:
burst_result = burst_detection(list_d, list_d_all, dates)
eva(dates, correct,　weighted_bursts)

SyntaxError: invalid character in identifier (<ipython-input-1-04eb9bcd2c2b>, line 2)

In [32]:
# burstラベルから期間を確認する
# dates[65:84]